In [4]:
import tkinter as tk
from tkinter import*
from tkinter.ttk import*
import tkinter.messagebox as tm
import pandas as pd
import datetime
import time
import requests
from io import StringIO
import sqlite3

def Update_price_daily():  #更新
    progress['value']=20    #進度條
    my_window.update()
    
    time.sleep(1)
    
    date_I_want = datetime.date.today()  #ymd用date  /   ymdhs用datetime
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date_I_want)  + '&type=ALLBUT0999')
    content = r.text.replace('=', '')
    lines = content.split('\n')
    lines = list(filter(lambda l:len(l.split('",')) > 10, lines))
    content = "\n".join(lines)
    if content == '': 
        return None
    
    progress['value']=50
    my_window.update()
    
    time.sleep(1)
    
    df = pd.read_csv(StringIO(content))
    df = df.astype(str)
    df['date'] = pd.to_datetime(date_I_want)  # 將爬取的日期存入 dataframe
    df = df.set_index(['證券代號', 'date','證券名稱'])  #將 證券代號 / date / 證券名稱 設定成index 
    df =df.iloc[:,0:9]  #取10列
    df = df[df.columns[df.isnull().all() == False]]   # 刪除不必要的欄位
    
    
    df.to_csv('Update_price_daily{:4}{:02}{:02}-Daily.csv'.format(datetime.date.today().year,datetime.date.today().month,datetime.date.today().day), encoding='utf_8_sig')
    df = pd.read_csv('Update_price_daily{:4}{:02}{:02}-Daily.csv'.format(datetime.date.today().year,datetime.date.today().month,datetime.date.today().day), index_col=['證券代號'])
    conn = sqlite3.connect('Update_price_daily{:4}{:02}{:02}-Daily.sqlite3'.format(datetime.date.today().year,datetime.date.today().month,datetime.date.today().day))
    df.to_sql('Update_price_daily{:4}{:02}{:02}-Daily.sqlite3'.format(datetime.date.today().year,datetime.date.today().month,datetime.date.today().day), conn, if_exists='replace')
    dffile='Update_price_daily{:4}{:02}{:02}-Daily.csv'.format(datetime.date.today().year,datetime.date.today().month,datetime.date.today().day)    
        
    time.sleep(1)
    progress['value']=100
    
    result_date="Update: Date {:4}.{:02}.{:02}-Time {:02}:{:02}:{:02}".format(datetime.datetime.today().year,
                                                                              datetime.datetime.today().month,
                                                                              datetime.datetime.today().day,
                                                                              datetime.datetime.today().hour,
                                                                              datetime.datetime.today().minute,
                                                                              datetime.datetime.today().second
                                                                              )
    tm.showinfo(title =str(result_date), message ='Finished.File name: '+ dffile)
    
    return df 

#---------------------------------------------------------------------------
    
my_window=tk.Tk()
my_window.geometry("600x400")
my_window.title("Welcome Lisa's World - Update stock price daily.")
my_window.iconbitmap('Update_stock_price_icon.ico')

#---------------------------------------------------------------------------

photo=tk.PhotoImage(file='Update_stock_price_image.png')
background_window=tk.Label(my_window,
                          text='== Welcome ==\nLet\'s update stock price now. ',
                          image=photo,
                          compound=tk.CENTER,
                          font=('Calibri',20,'bold italic'),
                          fg='black')
background_window.pack() 

#---------------------------------------------------------------------------

progress=tk.ttk.Progressbar(my_window,orient='horizontal',length=100,mode='determinate')
progress.pack()

#---------------------------------------------------------------------------

btnUpdate = tk.Button(my_window, text="Update Now (CSV & DB)", font="Calibri 14",command=Update_price_daily)
btnUpdate.pack(pady=12)


#---------------------------------------------------------------------------

my_window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\WanPing\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-52b5d0629bd2>", line 40, in Update_price_daily
    df.to_csv('Update_price_daily{:4}{:02}{:02}-Daily.csv'.format(datetime.date.today().year,datetime.date.today().month,datetime.date.today().day), encoding='utf_8_sig')
  File "C:\Users\WanPing\Anaconda3\lib\site-packages\pandas\core\generic.py", line 3020, in to_csv
    formatter.save()
  File "C:\Users\WanPing\Anaconda3\lib\site-packages\pandas\io\formats\csvs.py", line 157, in save
    compression=self.compression)
  File "C:\Users\WanPing\Anaconda3\lib\site-packages\pandas\io\common.py", line 424, in _get_handle
    f = open(path_or_buf, mode, encoding=encoding, newline="")
PermissionError: [Errno 13] Permission denied: 'Update_price_daily20191208-Daily.csv'
